In [1]:
import os

from tensorflow.python.keras.utils.np_utils import to_categorical

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

2.8.0


In [2]:
"""Initialization of data generator"""
from data_generator import DataGenerator
generator = DataGenerator(test_data=100, train_data=300)

In [51]:
"""Get train data"""
generator.generate_train_data()
train_data = generator.get_train_data()

In [52]:
"""Get test data"""
generator.generate_test_data()
test_data = generator.get_test_data()

In [9]:
"""Get shapes and examples of generated data"""
print(f'Train: {len(train_data)}, Test: {len(test_data)}')
print(f'Train ex.: {train_data[13]}')
print(f'Test ex.: {test_data[50]}')

Train: 900, Test: 100
Train ex.: RepresentedData(number=3, data=[1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])
Test ex.: RepresentedData(number=0, data=[0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0])


In [13]:
import random
n1 = random.randint(0, len(train_data) - 1)
print(n1)

405


In [14]:
# переделываем целевые значения
# y_train[i] -- это цифра, подлежащая различению, например 3
# Превратим её в массив [0 0 0 1 0 0 0 0 0 0]
# здесь 1 стоит на 3 месте. Это и есть целевые выходы
# нейронов последнего слоя, чтобы именно нейрон с номером 3
# откликался на изображение цифры 3
# Для такого преобразования есть специальная функция  to_categorical
# Такой формат называется вектор One-hot
print("Целевое значение до преобразования:", train_data[n1].number)

Целевое значение до преобразования: 5


In [15]:
train_data[n1].number = to_categorical(train_data[n1].number, 10)

In [16]:
print("Целевое значение после преобразования:", train_data[n1].number)


Целевое значение после преобразования: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [18]:
# Создаём модель слоя
class DenseNN(tf.Module):
    # Конструктор. Заполняет начальные значения
    def __init__(self, outputs, activate="relu"):
        super().__init__()
        self.outputs = outputs  # количество выходов = количеству нейронов в слое
        self.activate = activate  # тип активационной функции (по умолчанию relu)
        self.fl_init = False  # становится true после первой инициализации весов

    # Функция для расчёта выходных величин сети
    def __call__(self, x):
        if not self.fl_init:  # если весов ещё нет, создаём их случайными
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name="b")
            """Размер тензора w: (x.shape[-1], self.outputs)
            здесь x.shape[-1] - размер вектора входных данных,
            то есть, сколько входов в первый раз подадим на сеть,
            столько весов к каждому нейрону и сформируется
            следовательно, первый индекс - номер входа нейрона
            второй размер self.outputs совпадает с количеством выходов
            и с количеством нейронов в слое, значит второй
            индекс - номер нейрона
            stddev=0.1 - среднеквадратическое отклонение"""

            # преобразуем w и b в тензоры
            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.fl_init = True
        # матричное вычисление выхода слоя
        y = x @ self.w + self.b

        if self.activate == "relu":
            return tf.nn.relu(y)
        elif self.activate == "softmax":
            return tf.nn.softmax(y)

        return y


In [19]:
# создадим два полносвязных слоя со 128 нейронами и 10
layer_1 = DenseNN(128)
layer_2 = DenseNN(10, activate="softmax")


In [20]:
# Функция расчёта значений на выходе сети
def model_predict(x):
    y = layer_1(x)
    y = layer_2(y)
    return y  # layer_2(layer_1(x))


In [21]:
# Задание функции потерь
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))
# y_true, y_pred – это наборы one-hot векторов размером мини-батча


In [22]:
# определим оптимизатор для градиентного спуска
opt = tf.optimizers.Adam(learning_rate=0.001)

In [159]:
# Готовим обучение
BATCH_SIZE = 32
EPOCHS = 20
TOTAL = len(train_data)
print(f'Data amount in training set: {TOTAL}')

Data amount in training set: 300


In [25]:
# ОСТАЛОСЬ РЕФАКТОРИТЬ ПОСЛЕ ЭТОГО

In [146]:
from dataclasses import astuple

test_divided = list(zip(*[astuple(elem) for elem in test_data]))
x_test = test_divided[1]
y_test = test_divided[0]
print(len(y_test))

100


In [147]:
train_divided = list(zip(*[astuple(elem) for elem in train_data]))
x_train = train_divided[1]
y_train = train_divided[0]
print(len(y_train))

300


In [143]:
y_temp_train = to_categorical(y_train, 10)
y_temp_test = to_categorical(y_test, 10)

In [148]:
x_train = tf.reshape(tf.cast(x_train, tf.float32), [300, -1])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [100, -1])
# print("Новый размер: ", tf.shape(x_train).numpy())
# tf.data.Dataset - это класс с набором обучающих данных.
# Его можно создать из списков питона методом from_tensor_slices
# можно из файлов dataset = tf.data.TextLineDataset(["file1.txt", "file2.txt"])
# подробнее https://www.tensorflow.org/api_docs/python/tf/data/Dataset
print(len(x_train), len(y_train))
train_dataset = tf.data.Dataset.from_tensor_slices((list(x_train), list(y_temp_train)))
# [print(i) for i in train_dataset]
# Перемешиваем выборки и разбиваем на батчи
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

300 300


In [149]:
# for x_batch, y_batch in train_dataset:
#     print(x_batch, y_batch)

In [160]:
# Цикл обучения
for n in range(EPOCHS):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))

        loss += f_loss
        grads = tape.gradient(f_loss, [layer_1.trainable_variables, layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        opt.apply_gradients(zip(grads[1], layer_2.trainable_variables))

    print(f'Loss by epochs: {loss.numpy()}')

Loss by epochs: 0.12457847595214844
Loss by epochs: 0.1140364408493042
Loss by epochs: 0.10464748740196228
Loss by epochs: 0.09677798300981522
Loss by epochs: 0.08973123133182526
Loss by epochs: 0.08310197293758392
Loss by epochs: 0.07738358527421951
Loss by epochs: 0.07259823381900787
Loss by epochs: 0.06799490749835968
Loss by epochs: 0.0635729506611824
Loss by epochs: 0.06003269553184509
Loss by epochs: 0.05651690810918808
Loss by epochs: 0.05365120992064476
Loss by epochs: 0.05065663903951645
Loss by epochs: 0.04833006486296654
Loss by epochs: 0.04573696851730347
Loss by epochs: 0.04324094206094742
Loss by epochs: 0.04127754643559456
Loss by epochs: 0.03931944817304611
Loss by epochs: 0.03749953210353851


In [168]:
y = model_predict(x_test)
print(y, y_test)
y2 = tf.argmax(y, axis=1).numpy()
print(y2)

tf.Tensor(
[[0.2817147  0.04011312 0.01699745 0.10089666 0.1290632  0.11403999
  0.0628999  0.06201617 0.07735882 0.11490003]
 [0.0921929  0.03631851 0.1503857  0.2248195  0.12995201 0.0494606
  0.05183641 0.01574042 0.13041218 0.11888182]
 [0.10906884 0.00390123 0.14605759 0.0173611  0.04726188 0.00150001
  0.08189222 0.38820052 0.07019568 0.13456091]
 [0.4811356  0.01437907 0.22176035 0.035416   0.05436404 0.08430364
  0.03292378 0.06157504 0.00594252 0.00819997]
 [0.00443341 0.06052323 0.14655545 0.14605868 0.11441835 0.05344337
  0.07834858 0.01922002 0.18785769 0.18914121]
 [0.12907593 0.01815995 0.0055983  0.06138894 0.20234223 0.08377239
  0.04670025 0.12653767 0.30306092 0.02336341]
 [0.19958194 0.00500916 0.04521253 0.02929129 0.06611145 0.04869021
  0.3977377  0.03066827 0.03490917 0.1427883 ]
 [0.04452493 0.00383886 0.00932195 0.00781997 0.0388553  0.0233754
  0.04649417 0.16060175 0.61520153 0.04996614]
 [0.01000801 0.04165125 0.01436324 0.02090128 0.10923117 0.03340431
  0

In [165]:
print(y)
print(y_test)
print(y2)

tf.Tensor(
[[0.2817147  0.04011312 0.01699745 0.10089666 0.1290632  0.11403999
  0.0628999  0.06201617 0.07735882 0.11490003]
 [0.0921929  0.03631851 0.1503857  0.2248195  0.12995201 0.0494606
  0.05183641 0.01574042 0.13041218 0.11888182]
 [0.10906884 0.00390123 0.14605759 0.0173611  0.04726188 0.00150001
  0.08189222 0.38820052 0.07019568 0.13456091]
 [0.4811356  0.01437907 0.22176035 0.035416   0.05436404 0.08430364
  0.03292378 0.06157504 0.00594252 0.00819997]
 [0.00443341 0.06052323 0.14655545 0.14605868 0.11441835 0.05344337
  0.07834858 0.01922002 0.18785769 0.18914121]
 [0.12907593 0.01815995 0.0055983  0.06138894 0.20234223 0.08377239
  0.04670025 0.12653767 0.30306092 0.02336341]
 [0.19958194 0.00500916 0.04521253 0.02929129 0.06611145 0.04869021
  0.3977377  0.03066827 0.03490917 0.1427883 ]
 [0.04452493 0.00383886 0.00932195 0.00781997 0.0388553  0.0233754
  0.04649417 0.16060175 0.61520153 0.04996614]
 [0.01000801 0.04165125 0.01436324 0.02090128 0.10923117 0.03340431
  0

In [164]:
acc = y_test[y_temp_test == y2] / len(y_test) * 100
print(f'Summary accuracy: {acc}')

C:\Users\chdan\AppData\Local\Temp\ipykernel_16780\1261463277.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  acc = y_test[y_temp_test == y2] / y_test * 100


TypeError: unsupported operand type(s) for /: 'int' and 'tuple'